In [1]:
import os
import json
from datetime import datetime
import pandas as pd

In [2]:
# modificar segun entorno local
os.chdir( "/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"
folder_gmaps = "gmaps"
folder_reviews = "reviews_estados"

In [3]:
# extraemos nombres de cada estado segun nombre de carpeta
state_name = pd.DataFrame(
    pd.DataFrame(
        os.listdir(
            os.path.join(folder_data,folder_gmaps,folder_reviews))).rename(
                columns={0:"state"})["state"].str.split("-").str[1])
state_name.head(15)

,state
0,California
1,Pennsylvania


In [4]:
# indentificamos las columnas de interes y aquellas que se deben desanidar
vars_interes=[
    "gmap_id" # pk google maps
    ,"user_id" # pk user >> resp
    ,"name"
    ,"time"
    ,"rating"
    ,"text"
    #,"pics" # irrelevante
    #,"resp" # desanidar
    ]

vars_desanidar=[
    "gmap_id" # pk google maps
    ,"user_id" # pk user >> resp
    ,"time" # pk user >> resp
    ,"resp" # desanidar
    ]


In [5]:
# valor piloto;
# en el loop seria for i in range(min(state_name.index),max(state_name.index))
i = 1

# creamos una variable con el nombre de estado
# para facilitar el loop para todos los estados
state = state_name["state"][i]
folder_state = "review-" + str(state)

In [6]:
# contamos numero de archivos al interior de cada carpeta del estado
# para facilitar el loop
count_file = 0
# Iterate
for path in os.listdir(os.path.join(folder_data,folder_gmaps,folder_reviews,folder_state)):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder_data,folder_gmaps,folder_reviews,folder_state,path)):
        count_file += 1
count_file

16

In [7]:
# extraemos la informacion
data = []
for f in range(1, count_file):
    with open(
        os.path.join(folder_data,folder_gmaps,folder_reviews,folder_state, str(f) + ".json"), "r") as file:
        for line in file:
            data.append(json.loads(line))
gmaps_state = pd.DataFrame(data)
gmaps_state.info()
gmaps_state.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 137.3+ MB


,user_id,name,time,rating,text,pics,resp,gmap_id
0,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",None,None,0x89c6c63c8cd87141:0x54d0d283872eecbb
1,117609702832980320437,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...,None,None,0x89c6c63c8cd87141:0x54d0d283872eecbb


In [8]:
# nos concentramos en: vars_desanidar
gmaps_state_dsndr = pd.DataFrame(gmaps_state[vars_desanidar]).dropna(subset=["gmap_id","user_id"])
gmaps_state_dsndr.rename(
    columns={"time":"user_time"},
    inplace=True
    )
gmaps_state_dsndr.info()
gmaps_state_dsndr.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   gmap_id    object
 1   user_id    object
 2   user_time  int64 
 3   resp       object
dtypes: int64(1), object(3)
memory usage: 68.7+ MB


,gmap_id,user_id,user_time,resp
0,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,1517731762839,None
1,0x89c6c63c8cd87141:0x54d0d283872eecbb,117609702832980320437,1476276291163,None


In [9]:
# se normaliza la columna
gmaps_state_dsndr = gmaps_state_dsndr.set_index(["gmap_id","user_id","user_time"])
gmaps_state_dsndda = pd.json_normalize(
    gmaps_state_dsndr["resp"]).set_index(gmaps_state_dsndr.index) # type: ignore
gmaps_state_dsndda.reset_index(inplace=True) # mueve le indice a una columna
# renombramos para facilitar la union mas abajo
gmaps_state_dsndda.rename(
    columns={"time":"resp_time","text":"resp_text"},
    inplace=True
    )
gmaps_state_dsndda.info()
gmaps_state_dsndda.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   gmap_id    object 
 1   user_id    object 
 2   user_time  int64  
 3   resp_time  float64
 4   resp_text  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 85.8+ MB


,gmap_id,user_id,user_time,resp_time,resp_text
0,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,1517731762839,NaN,NaN
1,0x89c6c63c8cd87141:0x54d0d283872eecbb,117609702832980320437,1476276291163,NaN,NaN


In [10]:
# solo incluimos las variables de interes
gmaps_state_interes = gmaps_state[vars_interes].copy().dropna(subset=["gmap_id","user_id"])
# renombramos para facilitar la union mas abajo
gmaps_state_interes.rename(
    columns={"name":"user_name","time":"user_time","text":"user_text"},
    inplace=True
    )
gmaps_state_interes.info()
gmaps_state_interes.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   gmap_id    object
 1   user_id    object
 2   user_name  object
 3   user_time  int64 
 4   rating     int64 
 5   user_text  object
dtypes: int64(2), object(4)
memory usage: 103.0+ MB


,gmap_id,user_id,user_name,user_time,rating,user_text
0,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as..."
1,0x89c6c63c8cd87141:0x54d0d283872eecbb,117609702832980320437,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...


In [11]:
gmaps_state_interes[gmaps_state_interes.duplicated(keep=False)].head(4)

,gmap_id,user_id,user_name,user_time,rating,user_text
0,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as..."
1,0x89c6c63c8cd87141:0x54d0d283872eecbb,117609702832980320437,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...
2,0x89c6c63c8cd87141:0x54d0d283872eecbb,110563242018426632956,Rocky Kev,1338826945578,5,I usually give them a call before I stop by to...
3,0x89c6c63c8cd87141:0x54d0d283872eecbb,112895973505401399406,Josep Valls,1363286110554,5,My bike had been sitting outdoors for a good w...


In [12]:
# unir los dataframes
gmaps_state_norm = pd.merge(
    gmaps_state_interes,
    gmaps_state_dsndda,
    on=["gmap_id","user_id","user_time"]
    ,how="left"
    )

# generamos una columna state
# para facilitar la union de todos los datos
gmaps_state_norm["state"] = state

# movemos 'state' a la primera fila para facilitar la visualizacion
first_column = gmaps_state_norm.pop("state")
gmaps_state_norm.insert(0, "state", first_column)

gmaps_state_norm.info()
print(gmaps_state_norm.isna().sum())
gmaps_state_norm.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318268 entries, 0 to 2318267
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   state      object 
 1   gmap_id    object 
 2   user_id    object 
 3   user_name  object 
 4   user_time  int64  
 5   rating     int64  
 6   user_text  object 
 7   resp_time  float64
 8   resp_text  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 159.2+ MB
state              0
gmap_id            0
user_id            0
user_name          0
user_time          0
rating             0
user_text     980931
resp_time    2027274
resp_text    2027274
dtype: int64


,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text
0,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",NaN,NaN
1,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",NaN,NaN


In [13]:
# Convierte la columna 'user_time' a datetime y almacena el resultado en una nueva columna 'time_total'
gmaps_state_norm['user_time_year'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.year
gmaps_state_norm['user_time_month'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.month
gmaps_state_norm['user_time_day'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.day
gmaps_state_norm['user_time_hms'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.time
gmaps_state_norm['resp_time_year'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.year
gmaps_state_norm['resp_time_month'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.month
gmaps_state_norm['resp_time_day'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.day
gmaps_state_norm['resp_time_hms'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.time

gmaps_state_norm.info()
print(gmaps_state_norm.isna().sum())
gmaps_state_norm.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318268 entries, 0 to 2318267
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   state            object 
 1   gmap_id          object 
 2   user_id          object 
 3   user_name        object 
 4   user_time        int64  
 5   rating           int64  
 6   user_text        object 
 7   resp_time        float64
 8   resp_text        object 
 9   user_time_year   int32  
 10  user_time_month  int32  
 11  user_time_day    int32  
 12  user_time_hms    object 
 13  resp_time_year   float64
 14  resp_time_month  float64
 15  resp_time_day    float64
 16  resp_time_hms    object 
dtypes: float64(4), int32(3), int64(2), object(8)
memory usage: 274.1+ MB
state                    0
gmap_id                  0
user_id                  0
user_name                0
user_time                0
rating                   0
user_text           980931
resp_time          2027274
resp_text          2027274


,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text,user_time_year,user_time_month,user_time_day,user_time_hms,resp_time_year,resp_time_month,resp_time_day,resp_time_hms
0,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",NaN,NaN,2018,2,4,08:09:22.839000,NaN,NaN,NaN,NaT
1,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",NaN,NaN,2018,2,4,08:09:22.839000,NaN,NaN,NaN,NaT


In [14]:
gmaps_state_norm[gmaps_state_norm.duplicated(keep=False)].head(4)

,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text,user_time_year,user_time_month,user_time_day,user_time_hms,resp_time_year,resp_time_month,resp_time_day,resp_time_hms
0,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",NaN,NaN,2018,2,4,08:09:22.839000,NaN,NaN,NaN,NaT
1,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",NaN,NaN,2018,2,4,08:09:22.839000,NaN,NaN,NaN,NaT
2,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,117609702832980320437,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...,NaN,NaN,2016,10,12,12:44:51.163000,NaN,NaN,NaN,NaT
3,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,117609702832980320437,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...,NaN,NaN,2016,10,12,12:44:51.163000,NaN,NaN,NaN,NaT


In [15]:
gmaps_state_norm.drop_duplicates(inplace=True)

In [16]:
# df final por estado
gmaps_state_norm.to_parquet(os.path.join(
    folder_pipeline,folder_gmaps, str(folder_state) + "_norm.parquet"))
gmaps_state_norm.info()
print(gmaps_state_norm.isna().sum())
gmaps_state_norm.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 2216432 entries, 0 to 2318267
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   state            object 
 1   gmap_id          object 
 2   user_id          object 
 3   user_name        object 
 4   user_time        int64  
 5   rating           int64  
 6   user_text        object 
 7   resp_time        float64
 8   resp_text        object 
 9   user_time_year   int32  
 10  user_time_month  int32  
 11  user_time_day    int32  
 12  user_time_hms    object 
 13  resp_time_year   float64
 14  resp_time_month  float64
 15  resp_time_day    float64
 16  resp_time_hms    object 
dtypes: float64(4), int32(3), int64(2), object(8)
memory usage: 279.0+ MB
state                    0
gmap_id                  0
user_id                  0
user_name                0
user_time                0
rating                   0
user_text           971876
resp_time          1940713
resp_text          1940713
user_

,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text,user_time_year,user_time_month,user_time_day,user_time_hms,resp_time_year,resp_time_month,resp_time_day,resp_time_hms
0,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,104944742559079759337,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",NaN,NaN,2018,2,4,08:09:22.839000,NaN,NaN,NaN,NaT
2,Pennsylvania,0x89c6c63c8cd87141:0x54d0d283872eecbb,117609702832980320437,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...,NaN,NaN,2016,10,12,12:44:51.163000,NaN,NaN,NaN,NaT
